In [1]:
import requests

# API endpoint for text-davinci-003
endpoint = "https://api.openai.com/v1/engines/text-davinci-003/jobs"

# Your API key
api_key = "YOUR_API_KEY"

# Input text prompt
prompt = "Write a story about a person who goes on a journey."

# Request header
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Request payload
data = {
    "prompt": prompt,
    "max_tokens": 1000
}

# Make a request to the API endpoint
response = requests.post(endpoint, headers=headers, json=data)

# Check if the request was successful
if response.status_code == 200:
    # Extract the generated text from the response
    generated_text = response.json()["choices"][0]["text"]
    print(generated_text)
else:
    # Print an error message
    print("Failed to generate text. Response status code:", response.status_code)


Failed to generate text. Response status code: 401
